In [3]:
import openeo
import geopandas as gpd
import pandas as pd
import landuse_classification

In [4]:
con = openeo.connect("https://openeo.vito.be")

#### Target data 
LUCAS, 2018 (land use cover)

#### Input data
From S2: calculation of 7 indices (NDVI, NDMI, NDGI, ANIR, NDRE1, NDRE2, NDRE5) and keeping 2 bands (B06, B12)
From S1: VV, VH and VV/VH
For all of these, 10 features: p25, p50, p75, sd and 6 t-steps, with flexible range

#### Model
Random Forest, trained using custom hyperparameter, 70/30 split

In [ ]:
## User options
## TODO: @Bart Convert this into HTML widgets!


## % of test data
train_test_split = 0.3

algorithm = "RandomForest"

hyperparams = {
    num_trees: ##,
    mtry: ##
}

## USE ONLY S1, ONLY S2 or both
feature_raster = "both" # s1, s2 or both

## Area of interest
aoi = gpd.read_file('lucas/test_aoi.geojson')

## Stratification layer
strat_layer = gpd.read_file("lucas/test_stratification.geojson") #None or a geojson

## Easy lookup to see what classes represent
print(lookup_lucas["C10"])

## Custom legend
final_labels = {
    "Broadleaved woodland": ["C10"],
    "Coniferous woodland": ["C20"],
    "Mixed woodland": ["C30"],
    "Shrubland": ["D00", "D10", "D20"],
    "Grassland": ["E00", "E10", "E20", "E30"],
    "Cropland": ["B00", "B10", "B20", "B30","B40", "B50", "B70", "B80"],
    "Bare land": ["F00", "F10", "F20", "F40"],
    "Lichens and moss": ["F30"],
    "Built-up": ["A00", "A10", "A20", "A30"],
    "Inland water": ["G10", "G20", "G30"],
    "Sea and ocean": ["G40"],
    "Glaciers and permanent snow": ["G50"],
    "Wetlands": ["H00", "H10", "H20"]
}

## Cleaning & filtering operations on dataset
include_mixed_pixels = False
max_nr_samples = None
start_date = date(year,3,15)
end_date = date(year,10,31)
stepsize_s2 = 10
stepsize_s1 = 12

In [17]:
## TODO: @Bart use extended polygon LUCAS set https://essd.copernicus.org/articles/13/1119/2021/ and then extract points
data = gpd.read_file("https://artifactory.vgt.vito.be/auxdata-public/openeo/lucas.gpkg",mask=aoi)

if data.empty:
    raise ValueError("Your masked area is located outside of Europe or so small that no training data can be found within it")

In [50]:
## TODO: @Bart only keep label + geometry and make sure label matches general labels ([0:2] from label)
y = data[["LC1", "geometry"]]

In [ ]:
if strat_layer is None:
    strat_layer = aoi

In [ ]:
for stratum in strat_layer.iterfeatures():
    final_training_data = gpd.clip(data, gpd.GeoDataFrame(stratum))
    ## TODO: @Bart load_features spatial_extents toevoegen
    features, feature_list = load_features(feature_raster, stratum, start_date, end_date, stepsize_s2, stepsize_s1)

    ## TODO: @Bart testen tot aan hier!
    X = features.aggregate_spatial(geometries=y["geometry"], reducer=mean)
    ## dit testen tot hier
        
    ## TODO ? load_vectorcube, wellicht niet nodig gezien we alleen hoeven clippen naar een spatial extent. Load_files?
#     target = connection.load_vectorcube("x.geojson")
    
    ## TODO: fit_class_random_forest werkend krijgen voor X en y als inputs
    ml_model = con.fit_class_random_forest(predictors=X, target=y, training=100, **hyperparams)
    ml_model = ml_model.save_ml_model().start_and_wait().get_result()
    

In [ ]:
## Final inference: 2-3 tiles, 2-3 different scenarios
for stratum in strat_layer.iterfeatures():
    con.load_ml_model('batch_job_id')
    features, feature_list = load_features(feature_raster, stratum, start_date, end_date, stepsize_s2, stepsize_s1)

    def predict_rf(x: ProcessBuilder):
        ## TODO: bedenken hoe dit eruit gaat zien
        return x.predict_random_forest(features)

    y_pred = datacube.reduce_dimension(dimension="bands", process=predict_rf)

## Daarna verschillende predicties door verschillende strata weer terug mergen in 1 beeld ?
    
from sklearn.metrics import precision_recall_fscore_support
prec, rec, fscore, sup = precision_recall_fscore_support(y_test,y_pred)

In [ ]:
## Nice to have: custom rules toevoegen
#     ## Applying custom rules to the classification
#     def rule_glaciers(x: ProcessBuilder):
#         if x.is_nodata():
#             return array_create(data=[65534]*120)
#         return x    
#     datacube.apply_dimension(dimension="t", process=rule_glaciers)